In [ ]:
import pandas as pd
import re

In [ ]:
import kagglehub
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'medicaltranscriptions' dataset.
Path to dataset files: /kaggle/input/medicaltranscriptions


In [ ]:
dataset=pd.read_csv("/kaggle/input/medicaltranscriptions/mtsamples.csv")

In [ ]:
dataset.tail(1)

Unnamed: 0  \
4998        4998   

                                                                        description  \
4998   Acute allergic reaction, etiology uncertain, however, suspicious for Keflex.   

          medical_specialty                   sample_name  \
4998   Allergy / Immunology   Allergy Evaluation Consult    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
import re

def extract_uppercase_labels(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    # Reusing the regex from the basiccleaning function for uppercase labels
    pattern = r'(FINAL DIAGNOSES|CHIEF COMPLAINT|INDICATION|PROCEDURE|REASON FOR VISIT|HISTORY OF PRESENT ILLNESS|PAST MEDICAL HISTORY|SUBJECTIVE|DESCRIPTION|DIAGNOSIS|PREOPERATIVE DIAGNOSIS|2-D STUDY|CC)'
    labels = re.findall(pattern, text, re.IGNORECASE)
    return [label.upper() for label in labels]

# Apply the function to the 'transcription' column and collect all labels
all_labels = dataset['transcription'].apply(extract_uppercase_labels)

# Flatten the list of lists and get unique labels
unique_labels = sorted(list(set([label for sublist in all_labels for label in sublist])))

print("Unique uppercase labels found in 'transcription' column:")
for label in unique_labels:
    print(label)

Unique uppercase labels found in 'transcription' column:
2-D STUDY
CC
CHIEF COMPLAINT
DESCRIPTION
DIAGNOSIS
FINAL DIAGNOSES
HISTORY OF PRESENT ILLNESS
INDICATION
PAST MEDICAL HISTORY
PREOPERATIVE DIAGNOSIS
PROCEDURE
REASON FOR VISIT
SUBJECTIVE


In [ ]:
if "Unnamed: 0" in dataset.columns:
    dataset.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
def basiccleaning(text):
    if pd.isna(text):
        return ""
    text = str(text)

    text = re.sub(r'\s+', ' ', text) #extra space removing
    text = re.sub(r',\s*,+', ', ', text) #comma removing
    text = re.sub(r'([,.;:-])\1+', r'\1', text) # repeated punctuation removing
    text = re.sub(r'(INDICATION|FINAL DIAGNOSES|CHIEF COMPLAINT|INDICATION|PROCEDURE|REASON FOR VISIT|HISTORY OF PRESENT ILLNESS|PAST MEDICAL HISTORY|SUBJECTIVE|DESCRIPTION|DIAGNOSIS|PREOPERATIVE DIAGNOSIS|2-D STUDY|CC)\s*:?,?',
                  r'\1: ', text, flags=re.I) # Reformat specific uppercase labels to ensure they are followed by a colon and a space, preserving their original case.

    text = text.strip()
    return text

In [ ]:
dataset['description']=dataset['description'].apply(basiccleaning)
dataset['medical_specialty']=dataset['medical_specialty'].apply(basiccleaning)
dataset['sample_name']=dataset['sample_name'].apply(basiccleaning)
dataset['transcription']=dataset['transcription'].apply(basiccleaning)
dataset['keywords']=dataset['keywords'].apply(lambda x: basiccleaning(x) if isinstance(x, str) else "")

In [ ]:
def merge_fields(row):
    parts = [
        f"Sample Name: {row['sample_name']}",
        f"Medical Specialty: {row['medical_specialty']}",
        f"Description: {row['description']}",
        f"Transcription: {row['transcription']}",
        f"Keywords: {row['keywords']}"
    ]
    return "\n".join(parts)

In [ ]:
dataset["full_text"] = dataset.apply(merge_fields, axis=1)

In [ ]:
print(dataset["full_text"])

0       Sample Name: Allergic Rhinitis\nMedical Specia...
1       Sample Name: Laparoscopic Gastric Bypass Consu...
2       Sample Name: Laparoscopic Gastric Bypass Consu...
3       Sample Name: 2-D Echocardiogram - 1\nMedical S...
4       Sample Name: 2-D Echocardiogram - 2\nMedical S...
                              ...                        
4994    Sample Name: Chronic Sinusitis\nMedical Specia...
4995    Sample Name: Kawasaki Disease - Discharge Summ...
4996    Sample Name: Followup on Asthma\nMedical Speci...
4997    Sample Name: Asthma in a 5-year-old\nMedical S...
4998    Sample Name: Allergy Evaluation Consult\nMedic...
Name: full_text, Length: 4999, dtype: object


In [ ]:
import re
import pandas as pd

def deep_clean(text):
    text = str(text)

    # fix weird CSV commas
    text = re.sub(r',\s*,+', ', ', text)
    text = re.sub(r':\s*,', ': ', text)

    # fix \n
    text = text.replace('\\n', '\n')

    # fix spacing
    text = re.sub(r'\s+', ' ', text)



    return text.strip()


In [ ]:
dataset['full_text']=dataset['full_text'].apply(deep_clean)

In [ ]:
print(dataset['full_text'])

0       Sample Name: Allergic Rhinitis Medical Special...
1       Sample Name: Laparoscopic Gastric Bypass Consu...
2       Sample Name: Laparoscopic Gastric Bypass Consu...
3       Sample Name: 2-D Echocardiogram - 1 Medical Sp...
4       Sample Name: 2-D Echocardiogram - 2 Medical Sp...
                              ...                        
4994    Sample Name: Chronic Sinusitis Medical Special...
4995    Sample Name: Kawasaki Disease - Discharge Summ...
4996    Sample Name: Followup on Asthma Medical Specia...
4997    Sample Name: Asthma in a 5-year-old Medical Sp...
4998    Sample Name: Allergy Evaluation Consult Medica...
Name: full_text, Length: 4999, dtype: object


In [ ]:
import re

def clean_minimal(text):
    if not isinstance(text, str):
        return ""

    # 1. Convert escaped \n into real new lines
    text = text.replace("\\n", "\n")

    # 2. Remove repeated commas (CSV noise)
    text = re.sub(r",\s*,+", ", ", text)

    # 3. Fix CSV artifacts like ",MEDICATIONS:"
    text = re.sub(r",\s*([A-Z][A-Za-z ]+:)", r" \1", text)

    # 4. Remove cases like ": ," → ": "
    text = re.sub(r":\s*,", ": ", text)

    # 5. Normalize large whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [ ]:
dataset["full_text"] = dataset["full_text"].apply(clean_minimal)

In [ ]:
print(dataset['full_text'])

0       Sample Name: Allergic Rhinitis Medical Special...
1       Sample Name: Laparoscopic Gastric Bypass Consu...
2       Sample Name: Laparoscopic Gastric Bypass Consu...
3       Sample Name: 2-D Echocardiogram - 1 Medical Sp...
4       Sample Name: 2-D Echocardiogram - 2 Medical Sp...
                              ...                        
4994    Sample Name: Chronic Sinusitis Medical Special...
4995    Sample Name: Kawasaki Disease - Discharge Summ...
4996    Sample Name: Followup on Asthma Medical Specia...
4997    Sample Name: Asthma in a 5-year-old Medical Sp...
4998    Sample Name: Allergy Evaluation Consult Medica...
Name: full_text, Length: 4999, dtype: object


In [ ]:
!pip install langchain-text-splitters


Chunks and MetaDATA

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 80,
    separators = ["\n\n", "\n", ". ", "; ", ": ", " "]
)


In [ ]:
chunks = []
metadata_list = []

for idx, row in dataset.iterrows():
    text = row["full_text"]
    if not isinstance(text, str) or len(text.strip()) == 0:
        continue

    doc_chunks = text_splitter.split_text(text)

    for i, chunk in enumerate(doc_chunks):
        chunks.append(chunk)
        metadata_list.append({
            "doc_id": idx,
            "chunk_id": i,
            "sample_name": row.get("sample_name", ""),
            "medical_specialty": row.get("medical_specialty", ""),
            "description": row.get("description", ""),
            "keywords": row.get("keywords", ""),
            "source": "medical_transcriptions"
        })


In [ ]:
processed_chunks = pd.DataFrame({
    "chunk_text": chunks,
    "metadata": metadata_list
})

processed_chunks.to_json(
    "processed_medical_chunks.json",
    orient="records",
    indent=2
)

print("Saved processed_medical_chunks.json with", len(processed_chunks), "chunks.")


Saved processed_medical_chunks.json with 43420 chunks.


In [ ]:
import json
with open("processed_medical_chunks.json", "r") as f:
    chunks = json.load(f)

print(chunks[0])
print("Total chunks:", len(chunks))


{'chunk_text': 'Sample Name: Allergic Rhinitis Medical Specialty: Allergy / Immunology Description: A 23-year-old white female presents with complaint of allergies. Transcription: SUBJECTIVE: This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also', 'metadata': {'doc_id': 0, 'chunk_id': 0, 'sample_name': 'Allergic Rhinitis', 'medical_specialty': 'Allergy / Immunology', 'description': 'A 23-year-old white female presents with complaint of allergies.', 'keywords': 'allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,', 'source': 'medical_transcriptions'}}
Total chunks: 43420


Text Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = [c['chunk_text'] for c in chunks]
embs = model.encode(texts, show_progress_bar=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1357 [00:00<?, ?it/s]

In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 38.4 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np

dimension = embs.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(np.array(embs))
faiss.write_index(index, "medical_faiss.index")
